In [ ]:
# https://leetcode.com/problems/article-views-i/

import pandas as pd

def article_views(views: pd.DataFrame) -> pd.DataFrame:
    df = (views.loc[views["author_id"] == views["viewer_id"], ["author_id"]]
            .rename(mapper={"author_id": "id"}, axis=1)
            .sort_values(by="id")
            .drop_duplicates()
        )
    return df

# Subtle improvement

def article_views(views: pd.DataFrame) -> pd.DataFrame:
    df = (views.loc[views["author_id"] == views["viewer_id"], ["author_id"]]
            .rename(mapper={"author_id": "id"}, axis=1)
            .drop_duplicates()
            .sort_values(by="id")
        )
    return df

In [ ]:
# https://leetcode.com/problems/user-activity-for-the-past-30-days-i/

import pandas as pd

def user_activity(activity: pd.DataFrame) -> pd.DataFrame:
    period_mask = activity["activity_date"].between("2019-06-28", "2019-07-27", inclusive="both")
    df = (activity.loc[period_mask, ["activity_date", "user_id"]]
        .groupby("activity_date")
        .nunique()
        .reset_index()
        .rename(columns={"activity_date": "day", "user_id": "active_users"})
        )
    return df

In [ ]:
# https://leetcode.com/problems/sales-analysis-iii/

import pandas as pd

def sales_analysis(product: pd.DataFrame, sales: pd.DataFrame) -> pd.DataFrame:
    mask = sales["sale_date"].between("2019-01-01", "2019-03-31", inclusive="both")
    in_q = sales.loc[mask, "product_id"].unique()
    out_q = sales.loc[~mask, "product_id"].unique()
    res = product.loc[product["product_id"].isin(in_q[~ in_q.isin(out_q)]), ["product_id", "product_name"]]
    return res

In [ ]:
# https://leetcode.com/problems/project-employees-i/description/

import pandas as pd

def project_employees_i(project: pd.DataFrame, employee: pd.DataFrame) -> pd.DataFrame:
    df = project.merge(employee, on = "employee_id", how = "inner", suffixes = ("", ""))
    mean_xp = df.groupby("project_id")["experience_years"].mean().round(2).reset_index().rename(columns = {"experience_years": "average_years"})
    return mean_xp

# Lighter on memory

def project_employees_i(project: pd.DataFrame, employee: pd.DataFrame) -> pd.DataFrame:
    return (
        project[["project_id", "employee_id"]]
        .merge(
            employee[["employee_id", "experience_years"]],
            on="employee_id",
            how="inner",
            validate="many_to_one",   # each employee_id appears once in employee
        )
        .groupby("project_id", as_index=False)
        .agg(average_years=("experience_years", "mean"))
        .round({"average_years": 2})
    )

# Even lighter on memory avoiding full merge, using a map

def project_employees_i(project: pd.DataFrame, employee: pd.DataFrame) -> pd.DataFrame:
    xp = employee.set_index("employee_id")["experience_years"]
    return (
        project.assign(experience_years=project["employee_id"].map(xp))
               .dropna(subset=["experience_years"])        # inner-join semantics
               .groupby("project_id", as_index=False)["experience_years"].mean()
               .rename(columns={"experience_years": "average_years"})
               .round({"average_years": 2})
    )

In [ ]:
# https://leetcode.com/problems/department-highest-salary/submissions/1730219474/

import pandas as pd

def department_highest_salary(employee: pd.DataFrame, department: pd.DataFrame) -> pd.DataFrame:
    df = employee.merge(department, left_on = 'departmentId', right_on = 'id', suffixes = ('_employee', '_department'))
    max_df = df.groupby(by = 'name_department')['salary'].max()
    res_df = df.merge(max_df, on = 'name_department', suffixes = ('', '_max'))
    max_mask = res_df['salary'] == res_df['salary_max']
    return res_df[max_mask].rename(columns = {'name_department' : 'Department', 'name_employee' : 'Employee', 'salary' : 'Salary'})[['Department', 'Employee', 'Salary']]

# Of course you can just .loc and use 

def department_highest_salary(employee: pd.DataFrame, department: pd.DataFrame) -> pd.DataFrame:
    # Merging the employee and department DataFrames based on departmentId
    merged = employee.merge(department, left_on='departmentId', right_on='id', how='left')

    # Filtering employees whose salary equals the highest salary in their department
    highest_salary = merged.loc[merged.groupby('departmentId')['salary'].transform('max') == merged['salary']]
    
    # Renaming the columns to match the output format
    result = highest_salary[['name_x', 'salary', 'name_y']].rename(columns={ 
        'name_y': 'Department',  # department name column
        'name_x': 'Employee',    # employee name column
        'salary': 'Salary'       # salary column
    })
        
    # Returning the final result in the desired order
    return result[['Department', 'Employee', 'Salary']]

In [ ]:
# https://leetcode.com/problems/actors-and-directors-who-cooperated-at-least-three-times/

import pandas as pd

def actors_and_directors(df: pd.DataFrame) -> pd.DataFrame:
    coop_count = (df[["actor_id", "director_id"]].value_counts())
    multiple_coops = coop_count[coop_count >= 3].index.to_frame()
    return multiple_coops

In [ ]:
# https://leetcode.com/problems/swap-salary/

import pandas as pd

def swap_salary(salary: pd.DataFrame) -> pd.DataFrame:
    m_mask = salary["sex"] == "m"
    f_mask = salary["sex"] == "f"
    salary.loc[m_mask, "sex"] = "f"
    salary.loc[f_mask, "sex"] = "m"
    return salary

# more efficient solution

def swap_salary(salary: pd.DataFrame) -> pd.DataFrame:
    salary = salary.copy()
    salary.loc[:, "sex"] = salary.loc[:, "sex"].replace(to_replace = ["f", "m"], value = ["m", "f"])
    return salary

# Most efficient solution

# Possible approach

def swap_salary(salary: pd.DataFrame) -> pd.DataFrame:

    swap = lambda x: 'm' if x == 'f' else 'f'
    salary["sex"] = salary["sex"].apply(swap)

    return salary

In [ ]:
# https://leetcode.com/problems/biggest-single-number/solutions/

import pandas as pd

def biggest_single_number(df: pd.DataFrame) -> pd.DataFrame:
    return pd.DataFrame([df.set_index("num")[df["num"].value_counts() == 1].index.max()], columns = ["num"])

def biggest_single_number(df: pd.DataFrame) -> pd.DataFrame:
    unduped_mask = ~df.duplicated(keep = False)
    uniques = df[unduped_mask]
    max_unique = uniques.max()
    return pd.DataFrame({"num": max_unique})

def biggest_single_number(df: pd.DataFrame) -> pd.DataFrame:
    counts = df.groupby("num").size()
    singles = counts[counts == 1].index
    max_single = singles.max()
    return pd.DataFrame({"num": [max_single]}) 

In [ ]:
# https://leetcode.com/problems/product-sales-analysis-i/

import pandas as pd

def sales_analysis(sales: pd.DataFrame, product: pd.DataFrame) -> pd.DataFrame:
    df = pd.merge(sales, product, on = "product_id").loc[:, ["product_name", "year", "price"]]
    return df

In [ ]:
# https://leetcode.com/problems/nth-highest-salary/description/

import pandas as pd

def nth_highest_salary(employee: pd.DataFrame, N: int) -> pd.DataFrame:
    # 1) get unique salaries in descending order
    uniq = (
        employee
          .sort_values("salary", ascending=False)
          .drop_duplicates(subset=["salary"], keep="first")
    )
    
    # 2) pick the Nth (1-based) or NA if out of range
    if 1 <= N <= len(uniq):
        val = uniq.iloc[N-1]["salary"]
    else:
        val = pd.NA
    
    # 3) build the result DataFrame with the desired column name
    col_name = f"getNthHighestSalary({N})"
    return pd.DataFrame({col_name: [val]})


In [ ]:
# https://leetcode.com/problems/rank-scores/

import pandas as pd

def order_scores(scores: pd.DataFrame) -> pd.DataFrame:
    scores['rank'] = scores['score'].rank(method = 'dense', ascending = False)
    result_df = scores[['score', 'rank']].sort_values(by = 'score', ascending = False)
    return result_df

In [ ]:
# https://leetcode.com/problems/not-boring-movies/description/

import pandas as pd

def not_boring_movies(cinema: pd.DataFrame) -> pd.DataFrame:
    mask = ((cinema["id"] % 2 == 1) & (cinema["description"] != "boring"))
    return cinema[mask].sort_values("rating", ascending = False)

In [ ]:
# https://leetcode.com/problems/triangle-judgement/


import pandas as pd

def triangle_judgement(triangle: pd.DataFrame) -> pd.DataFrame:
    mask = triangle[["x", "y", "z"]].min(axis = 1) + triangle[["x", "y", "z"]].median(axis = 1) > triangle[["x", "y", "z"]].max(axis = 1)
    triangle["triangle"] = mask.map({True:"Yes", False:"No"})
    return triangle

In [ ]:
# https://leetcode.com/problems/sales-person/

import pandas as pd

def sales_person(sales_person: pd.DataFrame, company: pd.DataFrame, orders: pd.DataFrame) -> pd.DataFrame:
    sales_person_orders = pd.merge(sales_person, orders, on = "sales_id")
    sp_orders_company = pd.merge(sales_person_orders, company, on = "com_id", suffixes = ("_sp", "_o"))
    red_related = sp_orders_company[sp_orders_company["name_o"] == "RED"]
    return sales_person[ ~ sales_person["name"].isin(red_related["name_sp"])][["name"]]

# It is visible that filtering for "RED" could be done as a first measure to perhaps drastically reduce time on big dataframes

In [ ]:
# https://leetcode.com/problems/classes-more-than-5-students/

import pandas as pd

def find_classes(courses: pd.DataFrame) -> pd.DataFrame:
    df = courses.groupby(by = "class").count().reset_index()
    df = df[df["student"] >= 5]
    return df[["class"]]

In [ ]:
# https://leetcode.com/problems/big-countries/

import pandas as pd

def big_countries(world: pd.DataFrame) -> pd.DataFrame:
    big_countries = world[(world["area"] >= 3000000) | (world["population"] >= 25000000)]
    return big_countries[["name", "area", "population"]]

In [ ]:
# https://leetcode.com/problems/customer-placing-the-largest-number-of-orders/

import pandas as pd

def largest_orders(orders: pd.DataFrame) -> pd.DataFrame:
    order_counts = (
        orders
          .groupby(by = "customer_number")
          .size()
          .reset_index(name = "order_count")
    )
    top_customer = (
        order_counts
          .nlargest(1, "order_count")
    )[["customer_number"]]
    
    return top_customer

# Extremely elegant solution by https://leetcode.com/u/Kyrylo-Ktl/

def largest_orders(orders: pd.DataFrame) -> pd.DataFrame:
    return orders['customer_number'].mode().to_frame()

In [ ]:
# https://leetcode.com/problems/find-customer-referee/

import pandas as pd

def find_customer_referee(customer: pd.DataFrame) -> pd.DataFrame:
    result = customer[(customer['referee_id'] != 2) | customer['referee_id'].isnull()]
    return result[['name']]

In [ ]:
# https://leetcode.com/problems/employee-bonus/

import pandas as pd

def employee_bonus(employee: pd.DataFrame, bonus: pd.DataFrame) -> pd.DataFrame:
    df = pd.merge(employee, bonus, how = "left", on = "empId")
    
    df = df[(df["bonus"] < 1000) | df["bonus"].isnull()]
    return df[["name", "bonus"]]

In [ ]:
# https://leetcode.com/problems/combine-two-tables/

import pandas as pd


def combine_two_tables(person: pd.DataFrame, address: pd.DataFrame) -> pd.DataFrame:
    person_address = person.join(address.set_index('personId'), on = "personId")
    return person_address[["firstName", "lastName", "city", "state"]]